# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
ordersdf = pd.read_csv("Orders.csv")
ordersdf.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


In [17]:

spent_by_customer = ordersdf.groupby('CustomerID')['amount_spent'].agg('sum').reset_index()
spent_by_customer

,CustomerID,amount_spent
0,12346,77183.60
1,12347,4310.00
2,12348,1797.24
3,12349,1757.55
4,12350,334.40
...,...,...
4334,18280,180.60
4335,18281,80.82
4336,18282,178.05
4337,18283,2094.88


In [18]:
spent_by_customer.describe()


,CustomerID,amount_spent
count,4339.000000,4339.000000
mean,15299.936852,2053.793018
std,1721.889758,8988.248381
min,12346.000000,0.000000
25%,13812.500000,307.245000
50%,15299.000000,674.450000
75%,16778.500000,1661.640000
max,18287.000000,280206.020000


In [19]:


VIP = spent_by_customer['amount_spent'] > spent_by_customer['amount_spent'].quantile(0.95)
Preferred = (spent_by_customer['amount_spent'] > spent_by_customer['amount_spent'].quantile(0.75)) & (spent_by_customer['amount_spent'] < spent_by_customer['amount_spent'].quantile(0.95))

In [20]:
spent_by_customer['Client Type'] = pd.qcut(spent_by_customer['amount_spent'],[0, 0.75, 0.95, 1],labels= ['Regular', 'Preferred','VIP'])

In [21]:
spent_by_customer.head(10)

,CustomerID,amount_spent,Client Type
0,12346,77183.60,VIP
1,12347,4310.00,Preferred
2,12348,1797.24,Preferred
3,12349,1757.55,Preferred
4,12350,334.40,Regular
5,12352,2506.04,Preferred
6,12353,89.00,Regular
7,12354,1079.40,Regular
8,12355,459.40,Regular
9,12356,2811.43,Preferred


In [22]:
spent_by_customer['Client Type'].value_counts()

Regular      3254
Preferred     868
VIP           217
Name: Client Type, dtype: int64

---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [46]:
vipxcountry = pd.merge(left = spent_by_customer[['CustomerID','Client Type']],
                        right = ordersdf[['CustomerID', 'Country']],
                        how = 'left',
                        left_on = 'CustomerID',
                        right_on = 'CustomerID')
                       
vipxcountry.drop_duplicates('CustomerID', inplace = True)                      
vipxcountry = vipxcountry[vipxcountry['Client Type'] == 'VIP'].groupby('Country').agg({'CustomerID':'nunique'}).reset_index().sort_values('CustomerID', ascending = False)
vipxcountry

,Country,CustomerID
16,United Kingdom,177
7,Germany,10
6,France,9
15,Switzerland,3
13,Spain,2
11,Portugal,2
0,Australia,2
4,EIRE,2
8,Japan,2
5,Finland,1


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [54]:
vipandpref_countries = pd.merge(left = spent_by_customer[['CustomerID','Client Type']],
                        right = ordersdf[['CustomerID', 'Country']],
                        how = 'left',
                        left_on = 'CustomerID',
                        right_on = 'CustomerID')

vipandpref_countries = vipandpref_countries[vipandpref_countries['Client Type'] != 'Regular'].groupby('Country').agg({'CustomerID':'nunique'}).reset_index().sort_values('CustomerID', ascending = False)
vipandpref_countries

,Country,CustomerID
26,United Kingdom,932
10,Germany,39
9,France,29
2,Belgium,12
25,Switzerland,9
23,Spain,9
21,Portugal,7
19,Norway,7
14,Italy,5
8,Finland,5
